# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import numpy and pandas
import numpy as np
import pandas as pd 
import pymysql
from sqlalchemy import create_engine



# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset admissions predict in the Ironhack database.

In [41]:
# Your code
ip = "35.239.232.23"
login = "ironhacker_read"
password = "ir0nhack3r"
db = "admission_predict"

engine = create_engine(f'mysql+pymysql://{login}:{password}@{ip}/{db}')


In [42]:
admin_pre = pd.read_sql_query('SELECT * FROM admission_predict', engine)

In [43]:
admin_pre

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3,3.5,8,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.8
3,4,314,103,2,2,3,8.21,0,0.65
4,5,330,115,5,4.5,3,9.34,1,0.9
5,6,321,109,3,3,4,8.2,1,0.75
6,7,308,101,2,3,4,7.9,0,0.68
7,8,302,102,1,2,1.5,8,0,0.5
8,9,323,108,3,3.5,3,8.6,0,0.45
9,10,327,111,4,4,4.5,9,1,0.84


In [44]:
admin_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 9 columns):
Serial No.           385 non-null int64
GRE Score            385 non-null int64
TOEFL Score          385 non-null int64
University Rating    385 non-null int64
SOP                  385 non-null object
LOR                  385 non-null object
CGPA                 385 non-null object
Research             385 non-null int64
Chance of Admit      385 non-null object
dtypes: int64(5), object(4)
memory usage: 27.1+ KB


Remove trailing spaces at the end of the column names


In [45]:
# Your code
admin_pre.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [46]:
# Your code here:

admin_pre.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3,3.5,8,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.8
3,4,314,103,2,2,3,8.21,0,0.65
4,5,330,115,5,4.5,3,9.34,1,0.9


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [47]:
# Your code here:

admin_pre.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [48]:
admin_pre.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [49]:
# Your code here:
admin_pre.set_index('Serial No.', drop= False, inplace=True)


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [50]:
# Your code here:
admin_pre.set_index(['Serial No.', 'GRE Score', 'CGPA'], inplace= True)

admin_pre

,,,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
Serial No.,GRE Score,CGPA,,,,,,
1,337,9.65,118,4,4.5,4.5,1,0.92
2,316,8,104,3,3,3.5,1,0.72
3,322,8.67,110,3,3.5,2.5,1,0.8
4,314,8.21,103,2,2,3,0,0.65
5,330,9.34,115,5,4.5,3,1,0.9
6,321,8.2,109,3,3,4,1,0.75
7,308,7.9,101,2,3,4,0,0.68
8,302,8,102,1,2,1.5,0,0.5
9,323,8.6,108,3,3.5,3,0,0.45


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [51]:
# Your code here:

admin_pre.reset_index(inplace=True)

admin_pre

,Serial No.,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,1,337,9.65,118,4,4.5,4.5,1,0.92
1,2,316,8,104,3,3,3.5,1,0.72
2,3,322,8.67,110,3,3.5,2.5,1,0.8
3,4,314,8.21,103,2,2,3,0,0.65
4,5,330,9.34,115,5,4.5,3,1,0.9
5,6,321,8.2,109,3,3,4,1,0.75
6,7,308,7.9,101,2,3,4,0,0.68
7,8,302,8,102,1,2,1.5,0,0.5
8,9,323,8.6,108,3,3.5,3,0,0.45
9,10,327,9,111,4,4,4.5,1,0.84


In [52]:
admin_pre

,Serial No.,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,1,337,9.65,118,4,4.5,4.5,1,0.92
1,2,316,8,104,3,3,3.5,1,0.72
2,3,322,8.67,110,3,3.5,2.5,1,0.8
3,4,314,8.21,103,2,2,3,0,0.65
4,5,330,9.34,115,5,4.5,3,1,0.9
5,6,321,8.2,109,3,3,4,1,0.75
6,7,308,7.9,101,2,3,4,0,0.68
7,8,302,8,102,1,2,1.5,0,0.5
8,9,323,8.6,108,3,3.5,3,0,0.45
9,10,327,9,111,4,4,4.5,1,0.84


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [53]:
# Your code here:
admin_pre = admin_pre.astype({'CGPA': float})

In [54]:
admin_pre[(admin_pre['CGPA'] > 9) & (admin_pre['Research'] == 1)]


,Serial No.,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,1,337,9.65,118,4,4.5,4.5,1,0.92
4,5,330,9.34,115,5,4.5,3,1,0.9
10,11,328,9.10,112,4,4,4.5,1,0.78
19,20,328,9.50,116,5,5,5,1,0.94
20,21,334,9.70,119,5,5,4.5,1,0.95
21,22,336,9.80,119,5,4,3.5,1,0.97
28,29,338,9.40,118,4,3,4.5,1,0.91
29,30,340,9.60,114,5,4,4,1,0.9
30,31,331,9.80,112,5,4,5,1,0.94
31,32,320,9.20,110,5,5,5,1,0.88


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [55]:
# Your code here:
admin_pre = admin_pre.astype({'SOP' : float, 'Chance of Admit' : float})

In [56]:
admin_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 9 columns):
Serial No.           385 non-null int64
GRE Score            385 non-null int64
CGPA                 385 non-null float64
TOEFL Score          385 non-null int64
University Rating    385 non-null int64
SOP                  385 non-null float64
LOR                  385 non-null object
Research             385 non-null int64
Chance of Admit      385 non-null float64
dtypes: float64(3), int64(5), object(1)
memory usage: 27.1+ KB


In [57]:
admin_pre[(admin_pre['CGPA'] > 9) & (admin_pre['SOP'] < 3.5)][['Chance of Admit']].mean()


Chance of Admit    0.802
dtype: float64

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [58]:
def standardize_CGPA(col):
    # This function takes a column from a dataframe and returns a standardized column by subtracting the column's mean
    # and dividing by the column's standard deviation.
    
    # Your code here:
    mean = col.mean()
    std = col.std()
    admin_pre['CGPA_std'] = col.apply(lambda x : (x - mean) / std)

In [59]:
def standardize_GRE(col):
    # This function takes a column from a dataframe and returns a standardized column by subtracting the column's mean
    # and dividing by the column's standard deviation.
    
    # Your code here:
    mean = col.mean()
    std = col.std()
    admin_pre['GRE_std'] = col.apply(lambda x : (x - mean) / std)

In [60]:
def standardize_LOR(col):
    # This function takes a column from a dataframe and returns a standardized column by subtracting the column's mean
    # and dividing by the column's standard deviation.
    
    # Your code here:
    mean = col.mean()
    std = col.std()
    admin_pre['LOR_std'] = col.apply(lambda x : (x - mean) / std)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [61]:
# Your code here:
standardize_CGPA(admin_pre['CGPA'])

In [62]:
standardize_GRE(admin_pre['GRE Score'])

In [68]:
admin_pre = admin_pre.astype({'LOR' : float})

In [69]:
admin_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 11 columns):
Serial No.           385 non-null int64
GRE Score            385 non-null int64
CGPA                 385 non-null float64
TOEFL Score          385 non-null int64
University Rating    385 non-null int64
SOP                  385 non-null float64
LOR                  385 non-null float64
Research             385 non-null int64
Chance of Admit      385 non-null float64
CGPA_std             385 non-null float64
GRE_std              385 non-null float64
dtypes: float64(6), int64(5)
memory usage: 33.2 KB


In [70]:
standardize_LOR(admin_pre['LOR'])

In [71]:
admin_pre

,Serial No.,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,1,337,9.65,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647
1,2,316,8.00,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740
2,3,322,8.67,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166
3,4,314,8.21,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713
4,5,330,9.34,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713
5,6,321,8.20,109,3,3.0,4.0,1,0.75,-0.659198,0.369192,0.634193
6,7,308,7.90,101,2,3.0,4.0,0,0.68,-1.157218,-0.755462,0.634193
7,8,302,8.00,102,1,2.0,1.5,0,0.50,-0.991211,-1.274533,-2.153072
8,9,323,8.60,108,3,3.5,3.0,0,0.45,0.004829,0.542216,-0.480713
9,10,327,9.00,111,4,4.0,4.5,1,0.84,0.668856,0.888263,1.191647


In [87]:
admissions = admin_pre[['CGPA_std', 'GRE_std', 'LOR_std']]

In [88]:
admissions

,CGPA_std,GRE_std,LOR_std
0,1.747900,1.753381,1.191647
1,-0.991211,-0.063367,0.076740
2,0.121034,0.455704,-1.038166
3,-0.642597,-0.236391,-0.480713
4,1.233279,1.147798,-0.480713
5,-0.659198,0.369192,0.634193
6,-1.157218,-0.755462,0.634193
7,-0.991211,-1.274533,-2.153072
8,0.004829,0.542216,-0.480713
9,0.668856,0.888263,1.191647


We will generate the decision choice at random using the code below. Please run the cell.

In [89]:
# Run this code:

from random import choices
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [94]:
# Your code here:
decision_choice

['GRE_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'GR

In [96]:
decision_column = admissions.lookup(admissions.index, decision_choice)

In [99]:
admissions['decision_column'] = admissions.lookup(admissions.index, decision_choice)

/home/atwenter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
admissions

,CGPA_std,GRE_std,LOR_std,decision_column
0,1.747900,1.753381,1.191647,1.753381
1,-0.991211,-0.063367,0.076740,-0.991211
2,0.121034,0.455704,-1.038166,0.455704
3,-0.642597,-0.236391,-0.480713,-0.642597
4,1.233279,1.147798,-0.480713,-0.480713
5,-0.659198,0.369192,0.634193,-0.659198
6,-1.157218,-0.755462,0.634193,-0.755462
7,-0.991211,-1.274533,-2.153072,-0.991211
8,0.004829,0.542216,-0.480713,0.004829
9,0.668856,0.888263,1.191647,0.888263


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [102]:
# Your code here:

admissions['decision'] = admissions['decision_column'].apply(lambda x : 1 if x >0.8 else 0 )

/home/atwenter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [103]:
admissions

,CGPA_std,GRE_std,LOR_std,decision_column,decision
0,1.747900,1.753381,1.191647,1.753381,1
1,-0.991211,-0.063367,0.076740,-0.991211,0
2,0.121034,0.455704,-1.038166,0.455704,0
3,-0.642597,-0.236391,-0.480713,-0.642597,0
4,1.233279,1.147798,-0.480713,-0.480713,0
5,-0.659198,0.369192,0.634193,-0.659198,0
6,-1.157218,-0.755462,0.634193,-0.755462,0
7,-0.991211,-1.274533,-2.153072,-0.991211,0
8,0.004829,0.542216,-0.480713,0.004829,0
9,0.668856,0.888263,1.191647,0.888263,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [109]:
# Your code here:
admissions[admissions['decision'] == 1]['decision'].value_counts()


1    92
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [110]:
# Your code here:

admissions.columns

Index(['CGPA_std', 'GRE_std', 'LOR_std', 'decision_column', 'decision'], dtype='object')

In [125]:
admin_pre.columns

Index(['Serial No.', 'GRE Score', 'CGPA', 'TOEFL Score', 'University Rating',
       'SOP', 'LOR', 'Research', 'Chance of Admit', 'CGPA_std', 'GRE_std',
       'LOR_std'],
      dtype='object')

In [126]:
admin_pre.columns.str.lower().str.replace(' ', '_').str.replace('.', '')

Index(['serial_no', 'gre_score', 'cgpa', 'toefl_score', 'university_rating',
       'sop', 'lor', 'research', 'chance_of_admit', 'cgpa_std', 'gre_std',
       'lor_std'],
      dtype='object')

In [127]:
admin_pre.head()

,Serial No.,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,1,337,9.65,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647
1,2,316,8.00,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740
2,3,322,8.67,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166
3,4,314,8.21,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713
4,5,330,9.34,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713


In [130]:
admin_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 12 columns):
Serial No.           385 non-null int64
GRE Score            385 non-null int64
CGPA                 385 non-null float64
TOEFL Score          385 non-null int64
University Rating    385 non-null int64
SOP                  385 non-null float64
LOR                  385 non-null float64
Research             385 non-null int64
Chance of Admit      385 non-null float64
CGPA_std             385 non-null float64
GRE_std              385 non-null float64
LOR_std              385 non-null float64
dtypes: float64(7), int64(5)
memory usage: 36.2 KB


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [136]:
# Your code here:
adjusted_gre = pd.cut(admin_pre[admin_pre['University Rating'] >= 4]['GRE Score'].apply(lambda x : x + 10), 4)


In [137]:
adjusted_gre

0      (337.5, 350.0]
4      (337.5, 350.0]
9      (325.0, 337.5]
10     (337.5, 350.0]
18     (325.0, 337.5]
19     (337.5, 350.0]
20     (337.5, 350.0]
21     (337.5, 350.0]
22     (325.0, 337.5]
28     (337.5, 350.0]
29     (337.5, 350.0]
30     (337.5, 350.0]
31     (325.0, 337.5]
39     (337.5, 350.0]
40     (325.0, 337.5]
41     (325.0, 337.5]
42     (337.5, 350.0]
43     (337.5, 350.0]
45     (325.0, 337.5]
48     (337.5, 350.0]
49     (325.0, 337.5]
61     (325.0, 337.5]
65     (337.5, 350.0]
66     (337.5, 350.0]
67     (337.5, 350.0]
68     (325.0, 337.5]
69     (312.5, 325.0]
77     (337.5, 350.0]
78     (325.0, 337.5]
79     (325.0, 337.5]
            ...      
255    (325.0, 337.5]
256    (312.5, 325.0]
263    (337.5, 350.0]
271    (337.5, 350.0]
272    (337.5, 350.0]
273    (337.5, 350.0]
274    (325.0, 337.5]
275    (312.5, 325.0]
294    (325.0, 337.5]
296    (312.5, 325.0]
298    (337.5, 350.0]
299    (312.5, 325.0]
305    (325.0, 337.5]
314    (325.0, 337.5]
320    (31